<a href="https://colab.research.google.com/github/martykihn/xmas_movies_analysis/blob/main/spacy_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this code is just for Google Colab notebook word wrapping format

from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/TextFiles'

/content/drive/MyDrive/Colab Notebooks/TextFiles


In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
ls

 cached_lm_GPT2Tokenizer_126_text_as_list_test.csv
 cached_lm_GPT2Tokenizer_126_text_as_list_test.csv.lock
 cached_lm_GPT2Tokenizer_126_text_as_list_train.csv
 cached_lm_GPT2Tokenizer_126_text_as_list_train.csv.lock
 cached_lm_GPT2Tokenizer_126_text_nopunc.txt
 cached_lm_GPT2Tokenizer_126_text_nopunc.txt.lock
 data/
'google news corpus'/
 lda.model
 lda.model.expElogbeta.npy
 lda.model.id2word
 lda.model.state
 lda_tfidf.model
 lda_tfidf.model.expElogbeta.npy
 lda_tfidf.model.id2word
 lda_tfidf.model.state
 models/
 plots-finalformat.csv
 plots_test.txt
 plots_train.txt
 plots_valid.txt
 runs/
 story_generator_checkpoint/
 text_as_list.csv
 text_as_list_test.csv
 text_as_list_test_TOKENS.txt
 text_as_list_train.csv
 text_as_list_train_TOKENS.txt
 text_as_string_test.csv
 text_as_string_train.csv
 text_newlines.txt
 text_nopunc.txt
 text.txt
 transformers/


In [ ]:
plots = []
with open('/content/drive/MyDrive/Colab Notebooks/TextFiles/text_newlines.txt') as myfile:
  for plot in myfile:
    plot = plot.replace('\n', '')
    if plot == '':
      continue
    else:
      plots.append(str(plot))
myfile.close()

In [ ]:
len(plots)

260

In [ ]:
plots[:2]

['A ski-shop owner reluctantly moves himself, his wife, and his daughter in to an estate as live-in help for an elderly widow. While struggling to balance his career and family life, he has recurring dreams about an angel.',
 'Television movie updating Charles Dickens story, A Christmas Carol. Businesswoman Ebenita Scrooge treats her employees and customers poorly. She has no time for Christmas or the holiday spirit. On Christmas Eve, she is visited by the ghost of her dead partner Maude Marley and then by other spirits who remind her of her happy past and chronicle the bitterness and greed that have taken over her life. At last, she is shown her own death and funeral. No one is there to mourn her. This revelation shocks her into opening her heart and her checkbook.']

In [ ]:
# spacy needs input as a string not a list
plots_string = ' '.join(plots)

In [ ]:
plots_nlp = nlp(plots_string)

In [ ]:
i = 0
for entity in plots_nlp.ents:
  print(entity.text, entity.label_)
  i += 1
  if i > 10:
    break
  else:
    continue

Charles Dickens PERSON
A Christmas Carol WORK_OF_ART
Businesswoman Ebenita Scrooge PERSON
Christmas DATE
Christmas Eve DATE
Maude Marley PERSON
two CARDINAL
Bethlehem GPE
Christmas DATE
Christmas DATE
Santas NORP


In [ ]:
names = [i for i in plots_nlp.ents if i.label_.lower() in ['person']]

In [ ]:
norps = [i for i in plots_nlp.ents if i.label_.lower() in ['norp']]

In [ ]:
print(norps)

[Santas, Boyajian, Santas, Spelman, Christian, Californian, Santas, English, Evans, Carols, Ryans, Sean, Wills, Alaskan, Jules, Rossmans, Mountie, veterinarian, European, Amelia, Ryans, American, American, Italian, Jewish]


In [ ]:
len(names)

657

In [ ]:
names[:3]

[Charles Dickens, Businesswoman Ebenita Scrooge, Maude Marley]

In [ ]:
# return only first names
# use rule-based matcher to fine Name tokens (fil) w/ a space after
# and return only the token before the space
# [{ENT_TYPE": "PERSON"}, {"ORTH": " "}]

In [ ]:
names_list = [i.text for i in names]

In [ ]:
names_list[3]
# return only first name if there are two
firstnames_list = []
for i in range(len(names_list)):
  if " " in names_list[i]:
    firstname = names_list[i].split(' ')[0]
  else:
    firstname = names_list[i]
  firstnames_list.append(firstname)

In [ ]:
firstnames_list = [i for i in firstnames_list if i != 'The']

In [ ]:
firstnames_list[15:20]

['Kathleen', 'Noah', 'Max', 'Terry', 'Gary']

In [ ]:
len(firstnames_list)

656

In [ ]:
type(firstnames_list)

list

In [ ]:
firstnames_dict = {}
for name in firstnames_list:
  if name in firstnames_dict.keys():
    firstnames_dict[name] += 1
  else:
    firstnames_dict[name] = 1

In [ ]:
import pandas as pd

In [ ]:
df_names = pd.DataFrame.from_dict(firstnames_dict, orient='index', columns=['names'])

In [ ]:
df_names = df_names.sort_values(by='names', ascending=False)

In [ ]:
df_names[:20]

,names
Jack,18
Nick,16
Chris,12
Lucy,9
Joy,8
Maggie,8
Hannah,7
Charles,6
Jason,6
Mia,6


In [ ]:
df_names.to_csv('names_numbers.txt', header=True)

In [ ]:
# sub <MAN> <WOMAN> <SPIRIT> <SANTA>
# replace <Christmas> in the master text with nothing ...

In [ ]:
!pip install genderize

In [ ]:
from genderize import Genderize

In [ ]:
gender_list = Genderize().get(firstnames_list)

In [ ]:
type(gender_list)

list

In [ ]:
gender_list[4]

{'count': 8924, 'gender': 'female', 'name': 'Lauren', 'probability': 0.9}

In [ ]:
names_ = []
genders_ = []
for i in (range(len(gender_list))):
  names_.append(gender_list[i]['name'])
  genders_.append(gender_list[i]['gender'])

In [ ]:
names_[:4], genders_[:4]

(['Charles', 'Businesswoman', 'Maude', 'Nick'],
 ['male', 'female', 'female', 'male'])

In [ ]:
len(names_), len(genders_)

(656, 656)

In [ ]:
# create a dictionary for substitution with name as key & gender as value
gender_dict = {names_[i]: genders_[i] for i in range(len(names_))}

In [ ]:
list(gender_dict.items())[4:7]

[('Lauren', 'female'), ('Maggie', 'female'), ('Nathan', 'male')]

In [ ]:
# 'plots' is list of plots (as strings) - 260 of them
# substitute <GENDER> for name gender
# substitute <SANTA> for Santa
# remove <Christmas>
# look at <SPIRIT> ??

In [ ]:
plots_new = []
replacements = {'Christmas': '', '  ': ' ', 'Eve': '', 'Santa': '<SANTA>'}
for plot in plots:
  for key in replacements.keys():
    plot = plot.replace(key, replacements[key])
  plots_new.append(plot)

In [ ]:
plots_new[:2]

['A ski-shop owner reluctantly moves himself, his wife, and his daughter in to an estate as live-in help for an elderly widow. While struggling to balance his career and family life, he has recurring dreams about an angel.',
 'Television movie updating Charles Dickens story, A Carol. Businesswoman Ebenita Scrooge treats her employees and customers poorly. She has no time for or the holiday spirit. On , she is visited by the ghost of her dead partner Maude Marley and then by other spirits who remind her of her happy past and chronicle the bitterness and greed that have taken over her life. At last, she is shown her own death and funeral. No one is there to mourn her. This revelation shocks her into opening her heart and her checkbook.']

In [ ]:
# now do the gender replacement based on dictionary "gender_dict"
# first - change 'female' and 'male' to '<FEMALE>' and '<MALE>'
for k, v in gender_dict.items():
  try:
    gender_dict[k] = "<" + gender_dict[k].upper() + ">"
  except:
    continue

In [ ]:
#plots_new is list of plots (str)
#gender_dict is dictionary of name: gender (k/v)
temp = []
for plot in plots_new:
  try:
   tempx = ' '.join([gender_dict.get(i, i) for i in plot.split()])
   temp.append(tempx)
  except:
    continue
plots_new = temp.copy()

In [ ]:
with open('plots_new.txt', 'w') as f:
  for item in plots_new:
    f.write("%s\n" % item)

In [ ]:
# let's look at plots_new and look for other tokens for dictionaries
# ideas include: <SMALLTOWN> <HOMETOWN> and <BIGCITY>
# <FEMALE> <MALE> <OTHERMAN> <OTHERWOMAN>
# <PARENT> <CAREER> <CHILD>